# 思考问题1

1. 数据结构
    空间信息丢失：
    MLP需要将输入图像展平为一维向量（例如，3x28x28的RGB图像变为2352维向量），破坏了像素间的空间局部性（如相邻像素的关联性）。这种结构忽略了图像的2D/3D拓扑关系，导致模型难以捕捉边缘、纹理等局部模式。

    平移不变性缺失：
    图像中的目标无论位于哪个位置都应被识别，但MLP对输入顺序敏感（例如，平移后的图像会被视为完全不同的输入），需依赖大量数据隐式学习平移不变性，效率低下。

2. 参数量

    全连接的高计算成本：
    MLP的层间全连接导致参数量爆炸。例如，处理224x224的RGB图像时，输入层到第一隐藏层（假设隐藏层大小1024）的参数量为 224x224x3x1024 ≈ 154M，远超CNN的局部连接方式。

    过拟合风险：
    高参数量需要极大训练数据量来避免过拟合，而真实图像数据集通常标注成本高昂，难以满足MLP的需求。

3. 特征提取能力

    手工设计特征的局限性：
    传统MLP依赖人工设计特征（如SIFT+HOG+MLP组合），但图像的低级（边缘、角点）到高级特征（物体部件、整体）需分层自动提取。MLP缺乏层级归纳偏置，难以像CNN那样通过卷积核分层捕获特征。

    全局依赖与局部感知的矛盾：
    MLP理论上可通过深层网络拟合任意函数，但实际训练中难以同时兼顾局部细节（如纹理）和全局语义（如物体类别）。而CNN通过卷积-池化-深层的结构天然支持这一过程。

# 思考问题2
1. 局部感知

    - 操作方式：卷积核（如3x3）在图像上滑动，每次仅计算局部区域（如3x3像素块）的加权和，而非MLP的全连接。

    - 保留空间结构：像素的邻域关系（如边缘、纹理）被显式保留，避免展平操作导致的空间信息破坏。

2. 权重共享

    - 操作方式：同一卷积核在整个图像上共享参数，无论检测目标位于图像中心还是角落。

    - 平移不变性：相同模式（如猫耳）在不同位置会被同一核识别，无需MLP那样为每个位置学习独立参数。

3. 层次化特征提取

    - 低级→高级特征：

        - 浅层卷积：捕捉边缘、颜色等局部特征。

        - 深层卷积：组合局部特征为高级语义（如物体部件、整体类别）。

    - 空间信息传递：通过池化（如2x2 Max Pooling）逐步降低分辨率，但保留主要空间结构。

In [ ]:
from torch.nn import nn

class TwoLayerMLP(nn.model):

  def __init__(self,input_dim=3*32*32):
    super(TwoLayerMLP,self).__init__()
    self.flatten = nn.Flatten()

    # 第一层
    self.fc1 = nn.Linear(input_dim, 1024)
    self.bn1 = nn.BatchNorm1d(1024)

    # 第二层
    self.fc2 = nn.Linear(1024, 512)

    # 激活和Dropout
    self.relu = nn.ReLU()
    self.dropout = nn.Dropout(dropout_rate)

  def forward(self, x):
    x = self.flatten(x)

    # 第一层
    x = self.fc1(x)
    x = self.bn1(x)
    x = self.relu(x)
    x = self.dropout(x)

    # 第二层
    x = self.fc2(x)

    return x